In [1]:
import numpy as np # linear algebra
import pandas as pd
import sys

pd.options.display.max_rows = 99
pd.options.display.max_columns = 99

print('Python version: '+sys.version)
print('Numpy version: '+np.__version__)
print('Pandas version: '+pd.__version__)

Python version: 3.5.4 |Anaconda, Inc.| (default, Oct 15 2017, 03:43:24) [MSC v.1900 64 bit (AMD64)]
Numpy version: 1.14.5
Pandas version: 0.21.0


In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import statsmodels.api as sm
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import roc_curve, auc
import os, sys, time
from importlib import reload
path=os.getcwd()

In [3]:
df_train = pd.read_csv('sample_train.csv')
df_train.drop(labels=['Unnamed: 0'], axis=1, inplace=True)
print('train sample shape: '+str(df_train.shape))
df_test = pd.read_csv('sample_test.csv')
df_test.drop(labels=['Unnamed: 0'], axis=1, inplace=True)
print('test sample shape: '+str(df_test.shape))

train sample shape: (18463, 49)
test sample shape: (19214, 49)


In [3]:
from keras.layers import Input, Embedding, Concatenate, Reshape, BatchNormalization, LSTM, CuDNNLSTM, CuDNNGRU, Lambda
from keras.models import Model
from keras.layers.core import Dense, Activation, Dropout, RepeatVector
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras.initializers import Constant
import keras.backend as K


C:\Users\wsliu\AppData\Local\Continuum\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
from keras.layers import Dense, Embedding, Conv2D, MaxPool2D
from keras.layers import Reshape, Flatten, Dropout, Concatenate
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras.models import Model
#from sklearn.model_selection import train_test_split

In [6]:
n_DX = 29
n_PR = 15
DXs = ['DX'+str(n) for n in range(2,n_DX+2)]
PRs = ['PR'+str(n) for n in range(1,n_PR+1)]

In [7]:
# prepare data
df_train['y'] = 0
df_train.loc[df_train['readm30']==True, 'y'] = 1
df_test['y'] = 0
df_test.loc[df_test['readm30']==True, 'y'] = 1

X_cols = ['DX1', 'DX2', 'DX3', 'DX4', 'DX5', 'DX6', 'DX7', 'DX8', 'DX9', 'DX10',
          'DX11', 'DX12', 'DX13', 'DX14', 'DX15', 'DX16', 'DX17', 'DX18', 'DX19',
          'DX20', 'DX21', 'DX22', 'DX23', 'DX24', 'DX25', 'DX26', 'DX27', 'DX28',
          'DX29', 'DX30', 'PR1', 'PR2', 'PR3', 'PR4', 'PR5', 'PR6', 'PR7', 'PR8',
          'PR9', 'PR10', 'PR11', 'PR12', 'PR13', 'PR14', 'PR15', 'AGE', 'FEMALE', 'HOSP_NRD']

X_train = df_train[X_cols].values.astype(np.int)
Y_train = df_train[['y']].values.astype(np.int)
X_test = df_test[X_cols].values.astype(np.int)
Y_test = df_test[['y']].values.astype(np.int)

In [8]:
from setsum_layer import SetSum, MaskedSum, MaskedDense, MaskedPooling

In [9]:
code_embed_dim = 300
md_width = 5
fc_width = 512
hosp_embed_dim = 1
dropout = 0.3
job_index = 0
batchsize = 28
n_fold = 5
n_samples=100
n_disease=2135

In [11]:
embed_mat=np.random.uniform(0,1,size=[2135,code_embed_dim])
embed_initializer = Constant(np.random.uniform(0,1,size=[2135,code_embed_dim]))
skf = StratifiedKFold(n_splits=n_fold, random_state=24, shuffle=True)
trn_idx, val_idx = next(skf.split(X_train,Y_train))

In [13]:
train_df=df_train
DX1_array_train = train_df['DX1'].values
            
DX1_array_trn = DX1_array_train[trn_idx]
DX1_array_val = DX1_array_train[val_idx]

DX_mat_train = train_df[DXs].values
            
DX_mat_trn = DX_mat_train[trn_idx, :]
DX_mat_val = DX_mat_train[val_idx, :]

PR_mat_train = train_df[PRs].values
            
PR_mat_trn = PR_mat_train[trn_idx, :]
PR_mat_val = PR_mat_train[val_idx, :]

In [14]:
hosp_array_train = train_df['HOSP_NRD'].values
    
hosp_array_trn = hosp_array_train[trn_idx]
hosp_array_val = hosp_array_train[val_idx]
age_mean = train_df['AGE'].mean()
age_std = train_df['AGE'].std()
demo_mat_train = train_df[['AGE', 'FEMALE']].values
demo_mat_train[:, 0] = (demo_mat_train[:, 0]-age_mean)/age_std

In [15]:
other_mat_train = demo_mat_train

other_mat_trn = other_mat_train[trn_idx, :]
other_mat_val = other_mat_train[val_idx, :]

y_train = train_df.readm30.astype(int).values
Y_trn = to_categorical(y_train[trn_idx])
Y_val = to_categorical(y_train[val_idx])

In [16]:
model_name = 'setsum_nn'

In [17]:
numvar=45

In [21]:
hosp_cat = set(df_train['HOSP_NRD']).union(set(df_test['HOSP_NRD']))

In [19]:
from keras.engine.topology import Layer
from keras import objectives

def zero_loss(y_true, y_pred):
    return K.zeros_like(y_pred)


class CustomRegularization(Layer):
    def __init__(self, num_samples,num_classes, **kwargs):
        self.num_samples = num_samples
        self.num_classes = num_classes
        super(CustomRegularization, self).__init__(**kwargs)
        
    def build(self, input_shape):
        dense_shape, classes_shape = input_shape
        self.kernel = self.add_weight(name='kernel',
                                      shape=(self.num_classes, dense_shape[1]),
                                      initializer='uniform',
                                      trainable=True)
        self.bias = self.add_weight(name='bias',
                                    shape=(self.num_classes,),
                                    initializer='uniform',
                                    trainable=True) 
        super(CustomRegularization, self).build(input_shape)

    def call(self,x):
        inputs_in, labels_in = x
        labels = tf.manip.roll(labels_in,shift=1,axis=1)
        loss = tf.nn.sampled_softmax_loss(
            weights = self.kernel,
            biases = self.bias,
            labels = labels,
            inputs = inputs_in,
            num_sampled=self.num_samples,
            num_classes=self.num_classes,
            num_true=45)
        for i in range(2,45):
            labels = tf.manip.roll(labels_in,shift=i,axis=1)
            loss += tf.nn.sampled_softmax_loss(
            weights = self.kernel,
            biases = self.bias,
            labels = labels,
            inputs = inputs_in,
            num_sampled=self.num_samples,
            num_classes=self.num_classes,
            num_true=45)
            
        cost = tf.reduce_mean(loss)
        self.add_loss(cost,x)
        #you can output whatever you need, just update output_shape adequately
        #But this is probably useful
        return cost

    def compute_output_shape(self, input_shape):
        dense_shape,classes_shape = input_shape
        return (dense_shape[0],)

In [22]:
input_DX1 = Input(shape=(1,))
DX1_embed0 = Embedding(input_dim=embed_mat.shape[0], output_dim=code_embed_dim, embeddings_initializer=embed_initializer, 
                      name='DX1_embed')(input_DX1)
DX1_embed = Reshape((code_embed_dim,))(DX1_embed0)
input_DX = Input(shape = (len(DXs),))
DX_embed0 = Embedding(input_dim=embed_mat.shape[0], output_dim=code_embed_dim, mask_zero=False, embeddings_initializer=embed_initializer, 
                     name='DX_embed')(input_DX)
if model_name=='setsum_nn' or 'setsum_lr':
    DX_embed = MaskedDense(md_width, activation='relu')(DX_embed0)
    DX_embed = MaskedSum()(DX_embed0)
elif model_name=='embed_sum':
    DX_embed = MaskedSum()(DX_embed0)
elif model_name=='embed_pool':
    DX_embed = MaskedPooling()(DX_embed0)
input_PR = Input(shape = (len(PRs),))
PR_embed0 = Embedding(input_dim=embed_mat.shape[0], output_dim=code_embed_dim, mask_zero=False, embeddings_initializer=embed_initializer, 
                     name='PR_embed')(input_PR)
if model_name=='setsum_nn' or 'setsum_lr':
    PR_embed = MaskedDense(md_width, activation='relu')(PR_embed0)
    PR_embed = MaskedSum()(PR_embed0)
elif model_name=='embed_sum':
    PR_embed = MaskedSum()(PR_embed0)
elif model_name=='embed_pool':
    PR_embed = MaskedPooling()(PR_embed0)
input_hosp = Input(shape=(1,))
hosp_embed = Embedding(input_dim=len(hosp_cat), output_dim=hosp_embed_dim, input_length=1)(input_hosp)
hosp_embed = Reshape((hosp_embed_dim, ))(hosp_embed)
input_other = Input(shape=(other_mat_train.shape[1], ))
merged = Concatenate(axis=1)([DX1_embed, DX_embed, PR_embed, hosp_embed, input_other])
merged1 = Concatenate(axis=1)([DX1_embed0, DX_embed0, PR_embed0])
merged1 = Reshape((numvar*300,))(merged1)
merged2 = Concatenate(axis=1)([input_DX1,input_DX,input_PR])
merged2 = Reshape((45,))(merged2)
#merged3 = Concatenate(axis=2)([merged1,merged2])
cr =  CustomRegularization(n_samples,n_disease)([merged1,merged2])
if model_name=='setsum_nn':
    merged = Dense(fc_width, activation='relu')(merged)
    merged = Dropout(dropout)(merged)
prediction = Dense(2, activation='softmax')(merged)
model = Model(inputs=[input_DX1, input_DX, input_PR, input_hosp, input_other], outputs=[prediction,cr])

In [24]:
for l in model.layers:
    if l.name=='DX_embed' or l.name=='PR_embed':
        l.trainable = True

In [25]:
adam = Adam(lr=0.0002)
model.compile(optimizer=adam, loss=[K.categorical_crossentropy,zero_loss])

In [26]:
hist = model.fit([DX1_array_trn, DX_mat_trn, PR_mat_trn, hosp_array_trn, other_mat_trn], [Y_trn,np.random.randn(Y_trn.shape[0],1)], 
                     batch_size=batchsize, epochs=10, 
                     validation_data=[[DX1_array_val, DX_mat_val, PR_mat_val, hosp_array_val, other_mat_val], [Y_val,np.random.randn(Y_val.shape[0],1)]], 
                    verbose=1)

Train on 14770 samples, validate on 3693 samples
Epoch 1/10
  224/14770 [..............................] - ETA: 4:47:04 - loss: 228.1792 - dense_4_loss: 2.0867 - custom_regularization_2_loss: 0.0000e+00

KeyboardInterrupt: 